In [1]:
import instaloader
import pandas as pd
import requests
import json
import time
import re

C:\Users\admin\Anaconda3\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [2]:
# Set the path
path = 'C:/Users/admin/Documents/Python Files/flask'

In [3]:
def instagram(hashtag_csv):
    data = pd.read_csv(hashtag_csv)
    hashes = list(data['Hashtags'].values)
    elements = []
    for hashtag in hashes:
        url = 'https://www.instagram.com/explore/tags/'+hashtag+'/?__a=1'
        posts_json = requests.get(url).json()
        for i in range(len(posts_json['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges'])):
            post_caption = posts_json['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text']
            try:
                first_hashtag = re.findall(r"#(\w+)", post_caption)[0]
            except:
                first_hashtag = ''
            timestamp = posts_json['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges'][i]['node']['taken_at_timestamp']
            time_of_post = time.ctime(timestamp)
            num_of_comments = posts_json['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges'][i]['node']['edge_media_to_comment']['count']
            num_of_likes = posts_json['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges'][i]['node']['edge_liked_by']['count']
            user_id = posts_json['graphql']['hashtag']['edge_hashtag_to_top_posts']['edges'][i]['node']['owner']['id']
            L = instaloader.Instaloader()
            profile = instaloader.Profile.from_id(L.context, user_id)
            username = profile.username
            print(profile.username)
            profile_url = 'https://www.instagram.com/'+username+'/?__a=1'
            profile_json = requests.get(profile_url).json()
            followers_count = profile_json['graphql']['user']['edge_followed_by']['count']
            following_count = profile_json['graphql']['user']['edge_follow']['count']
            l = [str(hashtag),str(post_caption),str(first_hashtag),str(time_of_post),str(num_of_comments),str(num_of_likes),str(username),str(followers_count),str(following_count)]
            elements.append(l)
    my_df = pd.DataFrame(elements)
    my_df.to_csv(path+'/instagram_results.csv', index=True, header=['hashtag','post_caption','first_hashtag','time_of_post','num_of_comments','num_of_likes','username','followers_count','following_count'])

In [5]:
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
import pandas as pd

UPLOAD_FOLDER = path
ALLOWED_EXTENSIONS = set(['csv'])

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            instagram('C:/Users/admin/Documents/Python Files/flask/'+filename)
            return redirect(url_for('uploaded_file',
                                    filename='instagram_results.csv'))
    return '''
    <!doctype html>
    <title>Upload </title>
    <h1>Upload .csv File consisting of hashtags under 'Hashtags' Column name</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''
from flask import send_from_directory

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],
                               'instagram_results.csv')
from werkzeug import SharedDataMiddleware
app.add_url_rule('/uploads/<filename>', 'uploaded_file',
                 build_only=True)
app.wsgi_app = SharedDataMiddleware(app.wsgi_app, {
    '/uploads':  app.config['UPLOAD_FOLDER']
})
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2019 20:06:25] "GET / HTTP/1.1" 200 -
